In [1]:
# おまじない : パスの追加
import os
import sys
path = os.path.join(os.path.abspath(os.curdir), '../')
sys.path.append(path)

# twitterscraper

In [3]:
import datetime as dt
begin_date = dt.date(2019, 10, 5)
end_date = ｄｔ.date.today()
begin_date, end_date

(datetime.date(2019, 10, 5), datetime.date(2019, 10, 6))

In [2]:
from tweet_crawl import crawl_tweets

read config → ../config/config.ini


In [42]:
tweet_list = []
def crawled_callback(df_tweet):
    global tweet_list
    tweet_list.append(df_tweet)

In [43]:
%%time
crawl_tweets('ワンピース', begindate=begin_date, enddate=end_date, crawled_callback=crawled_callback)

2019-10-06 20:02:06,510 :tweet_crawl.py 49  :INFO    :crawl_tweets          :
cpu count : 4
pool size : 10
crawl : ワンピース, 2019-10-05 ~ 2019-10-06
day_per_loop : 10
平均用直近日数 : 3
1プール当たりのツイート割り当て : 1000

2019-10-06 20:02:06,523 :query.py   206 :DEBUG   :query_tweets          :queries: ['ワンピース since:2019-10-05 until:2019-10-06']
2019-10-06 20:02:06,527 :query.py   143 :DEBUG   :query_tweets_once_generator:Querying ワンピース since:2019-10-05 until:2019-10-06
2019-10-06 20:12:45,721 :query.py   151 :DEBUG   :query_tweets_once_generator:Got 19714 tweets for ワンピース%20since%3A2019-10-05%20until%3A2019-10-06.
2019-10-06 20:12:47,158 :tweet_crawl.py 132 :DEBUG   :_scrape_callback      :ワンピース since:2019-10-05 until:2019-10-06: got 19714（unique: 19694） tweets
2019-10-06 20:12:47,199 :tweet_crawl.py 125 :INFO    :_log_loop_report      :
ワンピース
Loop 1 : got 19714 tweets in 1 days (2019-10-05 ~ 2019-10-06)
total : 19714 tweets in 1 days crawled
1日当たりの平均ツイート数（total） : 19714
1日当たりの平均ツイート数（直近3日） : 19714
1プール当た

CPU times: user 2.18 s, sys: 350 ms, total: 2.53 s
Wall time: 10min 40s


In [13]:
len(tweet_list[0])

193

In [8]:
tweet_list[0][0]

{'html': '<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ja"><strong>ダンジョン飯</strong>8巻読んだー先が気になってたまらん</p>',
 'like_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'text': 'ダンジョン飯8巻読んだー先が気になってたまらん',
 'timestamp': Timestamp('2019-10-05 23:30:38'),
 'tweet_id': '1180626379898941440',
 'url': '/fkfkbody/status/1180626379898941440',
 'user_fullname': 'ぴろしき',
 'user_id': 'fkfkbody'}

# GetOldTweets3

In [1]:
import GetOldTweets3 as got

In [2]:
result = []
def receiveBuffer(resultsAux):
    global result
    result.extend(resultsAux)
    print(f'{len(resultsAux)}件取得しました。total {len(result)}件')

In [3]:
%%time
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('ダンジョン飯')\
                                           .setSince("2019-10-05")\
                                           .setUntil("2019-10-06")\
                                           .setLang('ja')
tweet_list_got = got.manager.TweetManager.getTweets(tweetCriteria, receiveBuffer=receiveBuffer)

100件取得しました。total 100件
100件取得しました。total 200件
34件取得しました。total 234件
CPU times: user 700 ms, sys: 70 ms, total: 770 ms
Wall time: 6.85 s


In [4]:
tweet_dict_list_got = [t.__dict__ for t in tweet_list_got]

In [5]:
import pandas as pd
df_unique_got = pd.DataFrame(tweet_dict_list_got).drop_duplicates('id', keep='first')

In [6]:
df_unique_got.shape

(234, 15)

In [15]:
df_unique_got.dtypes

{'author_id': dtype('int64'),
 'date': datetime64[ns, UTC],
 'favorites': dtype('int64'),
 'formatted_date': dtype('O'),
 'geo': dtype('O'),
 'hashtags': dtype('O'),
 'id': dtype('O'),
 'mentions': dtype('O'),
 'permalink': dtype('O'),
 'replies': dtype('int64'),
 'retweets': dtype('int64'),
 'text': dtype('O'),
 'to': dtype('O'),
 'urls': dtype('O'),
 'username': dtype('O')}

In [17]:
# df_unique_got.replace(None, None, regex=True)

In [23]:
_dict = df_unique_got.to_dict(orient='records')[0]

In [24]:
{k: v for }

{'author_id': 3410885292,
 'date': Timestamp('2019-10-05 23:58:37+0000', tz='UTC'),
 'favorites': 0,
 'formatted_date': 'Sat Oct 05 23:58:37 +0000 2019',
 'geo': '',
 'hashtags': '',
 'id': '1180633420717686784',
 'mentions': '',
 'permalink': 'https://twitter.com/Marsil_elf/status/1180633420717686784',
 'replies': 0,
 'retweets': 0,
 'text': '大体あんた誰なのよ 一体全体どういう',
 'to': None,
 'urls': '',
 'username': 'Marsil_elf'}

In [81]:
def extract_valid_records(_df, ser):
    return _df[~ser.isna() & ser.str.len() > 0]

# df_unique_got[~df_unique_got.mentions.isna() & df_unique_got.mentions.str.len() > 0]

In [85]:
extract_valid_records(df_unique_got, df_unique_got.urls)

,author_id,date,favorites,formatted_date,geo,hashtags,id,mentions,permalink,replies,retweets,text,to,urls,username
18,1153978131130097664,2019-10-05 23:58:46+00:00,0,Sat Oct 05 23:58:46 +0000 2019,,,1180633458592239618,,https://twitter.com/kuutan1652/status/11806334...,0,0,『犬服☆ペット服☆ワンピース☆ブルー(¥680)』 フリマアプリ「メルカリ」で販売中♪ ht...,None,https://item.mercari.com/jp/m53967282824/,kuutan1652
20,1327365146,2019-10-05 23:58:27+00:00,0,Sat Oct 05 23:58:27 +0000 2019,,,1180633380330717184,,https://twitter.com/one_piece_eigo/status/1180...,0,0,"""The act of not doubting."" That is ""strength!!...",None,http://twitpic.com/d70r9a,one_piece_eigo
100,1033276285475217409,2019-10-05 23:51:43+00:00,0,Sat Oct 05 23:51:43 +0000 2019,,,1180631687346024448,,https://twitter.com/LangSturrock/status/118063...,0,0,漫画アンテナ館 : 【ONEPIECE -ワンピース】スモーカー「ロギアの実を見付けたぞ！！...,None,http://mangamato.blog.jp/archives/19626569.html,LangSturrock
108,880076322,2019-10-05 23:50:54+00:00,0,Sat Oct 05 23:50:54 +0000 2019,,,1180631481246339072,,https://twitter.com/mymymero1/status/118063148...,0,0,夢のコラボ マイメロディー 半袖 マキシ ワンピース【MYMELODY マキシワンピース ワ...,None,http://ongi-mymelody.blogspot.jp/,mymymero1
109,852678522,2019-10-05 23:50:49+00:00,0,Sat Oct 05 23:50:49 +0000 2019,,#ヤフオク #札幌 #出張買取 #買取 #ワンピース,1180631459746349057,,https://twitter.com/ban_banko/status/118063145...,0,0,激レア！ ワンピース 一番くじ 麦わらの一味 第一弾 https://page.auctio...,None,https://page.auctions.yahoo.co.jp/jp/auction/k...,ban_banko
137,1078081510001991680,2019-10-05 23:48:52+00:00,0,Sat Oct 05 23:48:52 +0000 2019,,#rakuafl,1180630966592630784,,https://twitter.com/y9rYvkkw0QPFRIO/status/118...,0,0,【クーポン利用で3990→1999円】メール便 送料無料 キーネック ワッフル ワンピース ...,None,https://a.r10.to/hbmDc9,y9rYvkkw0QPFRIO
147,1018533129013743618,2019-10-05 23:48:16+00:00,0,Sat Oct 05 23:48:16 +0000 2019,,#ジャンププラスhttps,1180630817757810689,,https://twitter.com/midoumairu/status/11806308...,0,0,ヘッドスパ ジャンプの無料マンガアプリ「少年ジャンプ＋」で「[第65話]恋するワンピース」を...,None,https://shonenjumpplus.com/red/content/SHSA_JP...,midoumairu
148,951716828505518080,2019-10-05 23:48:10+00:00,0,Sat Oct 05 23:48:10 +0000 2019,,#rakuafl,1180630792130744320,,https://twitter.com/inamoriakemi/status/118063...,0,0,Vネック 花柄 マキシ 丈 シフォン ワンピース リゾート ワンピ ロング ドレス レディー...,None,https://a.r10.to/hzjSqb,inamoriakemi
184,265760136,2019-10-05 23:45:09+00:00,0,Sat Oct 05 23:45:09 +0000 2019,,,1180630033544568832,,https://twitter.com/avplaynow/status/118063003...,0,0,[セクシー]「白いワンピース 水谷美都」 http://idolvideom.blog.fc...,None,http://idolvideom.blog.fc2.com/blog-entry-1598...,avplaynow
195,14541750,2019-10-05 23:43:41+00:00,1,Sat Oct 05 23:43:41 +0000 2019,,#Amazon,1180629663552430080,,https://twitter.com/Reryu/status/1180629663552...,0,0,制服 ワンピース ロリータ コスプレ 制服 女子高生 女装 ロリータ服 大きいサイズ 森ガー...,None,https://www.amazon.co.jp/dp/B07XZ19WHL/ref=cm_...,Reryu


In [56]:
t = tweet_list_got_ja[0].__dict__

In [57]:
t

{'username': 'Marsil_elf',
 'to': None,
 'text': '大体あんた誰なのよ 一体全体どういう',
 'retweets': 0,
 'favorites': 0,
 'replies': 0,
 'id': '1180633420717686784',
 'permalink': 'https://twitter.com/Marsil_elf/status/1180633420717686784',
 'author_id': 3410885292,
 'date': datetime.datetime(2019, 10, 5, 23, 58, 37, tzinfo=datetime.timezone.utc),
 'formatted_date': 'Sat Oct 05 23:58:37 +0000 2019',
 'mentions': '',
 'hashtags': '',
 'geo': '',
 'urls': ''}

In [63]:
df_unique_got.groupby('username').count().sort_values('author_id', ascending=False)

,author_id,date,favorites,formatted_date,geo,hashtags,id,mentions,permalink,replies,retweets,text,to,urls
username,,,,,,,,,,,,,,
october3347,153,153,153,153,153,153,153,153,153,153,153,153,0,153
Anju82555658,106,106,106,106,106,106,106,106,106,106,106,106,97,106
bGPD70pzBEwyyeq,77,77,77,77,77,77,77,77,77,77,77,77,52,77
satuki45440,58,58,58,58,58,58,58,58,58,58,58,58,47,58
anisogo,53,53,53,53,53,53,53,53,53,53,53,53,0,53
majo_orochi_,51,51,51,51,51,51,51,51,51,51,51,51,18,51
reo2145,49,49,49,49,49,49,49,49,49,49,49,49,0,49
Guitarbase71Bot,48,48,48,48,48,48,48,48,48,48,48,48,0,48
onepeeceBOT,48,48,48,48,48,48,48,48,48,48,48,48,0,48


In [47]:
len(tweet_list[0]), len(tweet_list_got)

(19694, 19699)

In [32]:
tweet_list_got_ja[3].__dict__

{'username': 'toiwata',
 'to': None,
 'text': 'ダンジョン飯を1巻から読み直して、最新刊の終盤まで来たが、次の巻が出るのを一旦待つことにする。(主人公の運命や如何に)',
 'retweets': 0,
 'favorites': 0,
 'replies': 0,
 'id': '1180624846553677824',
 'permalink': 'https://twitter.com/toiwata/status/1180624846553677824',
 'author_id': 72292046,
 'date': datetime.datetime(2019, 10, 5, 23, 24, 33, tzinfo=datetime.timezone.utc),
 'formatted_date': 'Sat Oct 05 23:24:33 +0000 2019',
 'mentions': '',
 'hashtags': '',
 'geo': '',
 'urls': ''}

In [30]:
tweet_list[0][2]['text']

'ダンジョン飯、古本でも売ってるけど私は新本で買うぞ！！本は……基本新本で買うと決めたのだ！！'